<a href="https://colab.research.google.com/github/Pibastte/AssistanceFraudeFeachy/blob/main/TikTakToe_Base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup of the game

In [ ]:
import math
import time
from random import randint
from numba import njit, prange

In [ ]:
repr = {
    -1: "X",
    1: "O",
    0: "-"
}

In [ ]:
result = {
    -1: "X",
    1: "O",
    0: "tie"
}

In [ ]:
ai = -1
player = 1

In [ ]:
def print_board(board):
  print("  " + " ".join(str(x) for x in list(range(len(board)))))
  for i in range(len(board)):
    print(str(i), " ".join([repr[x] for x in board[i]]))

In [ ]:
def is_patern(board, i, j, player, direction, n_to_win, n=1):
  #toggleprint("ispatern", i, j, n)
  # If one player has won
  if n == n_to_win:
    #toggleprint("won")
    return player
  # If the current direction is not of length 4
  elif board[i][j] == 0:
    return 0
  
  if board[i][j] == player and direction == "r" and len(board[0]) - j >= n_to_win - n:
    #toggleprint("r", i, j, n)
    return is_patern(board, i, j+1, board[i][j], direction, n_to_win, n+1)
  if board[i][j] == player and direction == "ddr" and len(board[0]) - j >= n_to_win - n and len(board) - i >= n_to_win - n:
    #toggleprint("ddr", i, j, n)
    return is_patern(board, i+1, j+1, board[i][j], direction, n_to_win, n+1)
  if board[i][j] == player and direction == "dur" and len(board[0]) - j >= n_to_win - n and i >= n_to_win - n - 1:
    #toggleprint("dur", i, j, n)
    return is_patern(board, i-1, j+1, board[i][j], direction, n_to_win, n+1)
  if board[i][j] == player and direction == "d" and len(board) - i >= n_to_win - n:
    #toggleprint("d", i, j, n)
    return is_patern(board, i+1, j, board[i][j], direction, n_to_win, n+1)

def winner(board, n_to_win = 4):
  finished = True
  for i in range(len(board)):
    for j in range(len(board[0])):
      finished = finished and board[i][j] != 0
      if board[i][j] != 0:
        #toggleprint("found", i, j)
        right = 0
        diag_up_right = 0
        diag_down_right = 0
        down = 0
        # Si l'y a la place à droite
        if len(board[0]) - j >= n_to_win:
          #toggleprint("place à droite")
          right = is_patern(board, i, j+1, board[i][j], "r", n_to_win)
          # Si l'y a la place en haut
          if i >= n_to_win - 1:
            diag_up_right = is_patern(board, i-1, j+1, board[i][j], "dur", n_to_win)
          # Si l'y a la place en bas
          if len(board) - i >= n_to_win:
            diag_down_right = is_patern(board, i+1, j+1, board[i][j], "ddr", n_to_win)
        # Si l'y a la place en bas
        if len(board) - i >= n_to_win:
          #toggleprint("place en bas")
          down = is_patern(board, i+1, j, board[i][j], "d", n_to_win)

        if right == 1 or right == -1:
          return right
        if diag_up_right == 1 or diag_up_right == -1:
          return diag_up_right
        if diag_down_right == 1 or diag_down_right == -1:
          return diag_down_right
        if down == 1 or down == -1:
          return down
  if finished == True:
    return 0
  else:
    return None

In [ ]:
def update_mtt_positions(i, j, mtt, size, n_to_win=4):
  #print(type(mtt))
  if mtt["top"] == None or mtt["bottom"] == None or mtt["left"] == None or mtt["right"] == None:
    mtt["top"] = max(i - 3, 0)
    mtt["left"] = max(j - 3, 0)
    mtt["right"] = min(j + 3, n_to_win + 2 + mtt["left"], size - 1)
    mtt["bottom"] = min(i + 3, n_to_win + 2 + mtt["top"], size - 1)
  else:
    if i - 1 <= mtt["top"]:
      mtt["top"] = max(i - 2, 0)
    elif i + 1 >= mtt["bottom"]:
      mtt["bottom"] = min(i + 2, size - 1)
    if j - 1 <= mtt["left"]:
      mtt["left"] = max(j - 2, 0)
    elif j + 1 >= mtt["right"]:
      mtt["right"] = min(j + 2, size - 1)
  
  print("new mtt: ({}, {}), ({}, {})".format(mtt["top"], mtt["left"], mtt["bottom"], mtt["right"]))
  return mtt

def play_tiktaktoe(nbr_tours = 0, alphabeta=True, size = 4):
  
  # To reduce the size of the board especially in the beginning of games, we track the furthest values on each side of the grid.
  # Then we can only focus on the board 1 square wider
  mtt = {
      "top": None, # most to the top
      "bottom": None, # most to the bottom
      "left": None, # most to the left
      "right": None # moste to the right
  }

  print(type(mtt))

  board = [[0 for i in range(size)] for j in range(size)]

  print("Tour " + str(nbr_tours))
  print_board(board)
  while (True):
    # Si un des joueurs a gagné
    has_won = winner(board, n_to_win=min(size, 4))
    #print(has_won)
    if has_won != None:
      print(result[has_won] + " has won") if has_won != 0 else print("It's a TIE !")
      return has_won
    
    # Si personne n'a gagné on continue à jouer (X commence)
    print("Tour " + str(nbr_tours))
    print_board(board)

    if nbr_tours % 2 == 0: # C'est au tour de l'IA
      if nbr_tours == 0:
        i,j = (size-1)//2, (size-1)//2
        board[i][j] = ai
        mtt = update_mtt_positions(i, j, mtt=mtt, size=len(board), n_to_win=min(size, 4))
      else:
        # c'est au tour de l'IA (X)
        i, j = get_best_move(board=board, alphabeta=alphabeta, mtt=mtt)
        board[i][j] = ai
        mtt = update_mtt_positions(i, j, mtt=mtt, size=len(board), n_to_win=min(size, 4))
      
    else:
      # c'est au tour du joueur (O)
      i, j = None, None
      first = True
      while ((i == None and j == None) or board[i][j] != 0):
        if first != True:
          print("Please enter a valid position")
        i = int(input("entrez un nombre entre 0 et "+ str(size -1) ))
        j = int(input("entrez un nombre entre 0 et " + str(size -1)))
        first = False
      board[i][j] = player
      mtt = update_mtt_positions(i, j, mtt=mtt, size=len(board), n_to_win=min(size, 4))

    # Check if point (i,j) is outside of the delimited space
    print("mtt:", mtt)
    nbr_tours += 1



# Minmax algorithm

In [ ]:
# AI -> X
# opponent -> O

def best_coordinates(board, best_scores, best_depths):
  m = max(best_scores)
  indices = [i for i, x in enumerate(best_scores) if x == m]
  print(indices)
  min_depth = best_depths[indices[0]]
  max_i = indices[0]
  for i in indices:
    if best_depths[i] < min_depth:
      print(i)
      min_depth = best_depths[i]
      max_i = i
  print("best_coordinates:",max_i//len(board), max_i%len(board))
  return max_i//len(board), max_i%len(board)

# Returns the best possible move that the AI can make
def get_best_move(board, max_depth=4, alphabeta = True, n_to_win = 4, mtt=None):
  #print("max depth:", max_depth)
  #print(mtt)
  best_scores = [-math.inf] * (len(board))**2
  best_depths = [math.inf] * (len(board))**2

  start_time = time.time()
  print("bottom", mtt["bottom"], "right", mtt["right"])
  for i in range(mtt["top"], mtt["bottom"] - n_to_win):
    for j in range(mtt["left"], mtt["right"] - n_to_win):
      print(i,j)
      for inner_i in range(0, n_to_win + 2):
        for inner_j in range(0, n_to_win + 2):
          curr_i, curr_j = i + inner_i, j + inner_j
          #print(i, j, inner_i, inner_j)
          if board[curr_i][curr_j] == 0: # Iterates over all available positions
            temp_mtt = {
                "top": i,
                "left": j,
                "bottom": i + n_to_win + 1,
                "right": j + n_to_win + 1
            }
            #print(temp_mtt)
            board[curr_i][curr_j] = ai  # Plays (i,j) position
            score, depth = minmax(board, 0, -math.inf, math.inf, False,  max_depth=max_depth, alphabeta=alphabeta, n_to_win=n_to_win, mtt=temp_mtt) # Gets the score of the (i,j) position
            print(curr_i, curr_j, score, depth)
            board[curr_i][curr_j] = 0 # Unplays the (i,j) position
            if score == 1 and depth == 0:
              return curr_i, curr_j
            if score == -1:
              best_scores[len(board) * curr_i + curr_j] = -1
              best_depths[len(board) * curr_i + curr_j] = depth
            if score > best_scores[len(board) * curr_i + curr_j] and best_scores[len(board) * curr_i + curr_j] != -1:  # If the score is better we keep the position
              best_scores[len(board) * curr_i + curr_j] = score
              best_depths[len(board) * curr_i + curr_j] = depth
            elif score == best_scores[len(board) * curr_i + curr_j] and depth < best_depths[len(board) * curr_i + curr_j] and best_scores[len(board) * curr_i + curr_j] != -1:  # Optimizes for faster winning
              best_scores[len(board) * curr_i + curr_j] = score
              best_depths[len(board) * curr_i + curr_j] = depth
  print(time.time() - start_time, "seconds to play")
  return best_coordinates(board, best_scores, best_depths)

def minmax(board, depth, alpha, beta, is_maximizing, max_depth, alphabeta = True, n_to_win=4, mtt=None):
  has_won = winner(board, n_to_win)
  if has_won != None:   # Stop condition of the recursion
    # print("LOST")
    # print_board(board)
    # if has_won == 0:
    #   print("DRAW")
    return -1 * has_won, depth

  if is_maximizing:   # If we are maximizing
    best_score = -math.inf
    best_depth = math.inf

    for i in range(mtt["top"], mtt["bottom"]+1):
      for j in range(mtt["left"], mtt["right"]+1):
        if board[i][j] == 0:  # Checks for all available spots
          board[i][j] = ai
          if depth <= max_depth:
            score, temp_depth = minmax(board, depth + 1, alpha, beta, False, max_depth, alphabeta=alphabeta, mtt=mtt)
          else:
            score, temp_depth = 0, math.inf
          board[i][j] = 0
          best_score = max(best_score, score)
          if score == best_score and temp_depth < best_depth:
            best_depth = temp_depth

          alpha = max(alpha, score)
          if alpha >= beta and alphabeta:   # Implementation du alpha beta
            break
    return best_score, best_depth
  else:
    best_score = math.inf
    best_depth = math.inf

    for i in range(mtt["top"], mtt["bottom"]+1):
      for j in range(mtt["left"], mtt["right"]+1):
        if board[i][j] == 0:    # Checks for all available spots
          board[i][j] = player
          if depth <= max_depth:
            score, temp_depth = minmax(board, depth + 1, alpha, beta, True, max_depth, alphabeta=alphabeta, mtt=mtt)
          else:
            score, temp_depth = 0, math.inf
          board[i][j] = 0
          best_score = min(score, best_score)
          if best_score == score and temp_depth < best_depth:
            best_depth = temp_depth
          beta = min(beta, score)
          if beta <= alpha and alphabeta:  # Implementation du alpha beta
            break
    return best_score, best_depth

  

#Time to play

In [ ]:
play_tiktaktoe(alphabeta=True, size=6)

<class 'dict'>
Tour 0
  0 1 2 3 4 5
0 - - - - - -
1 - - - - - -
2 - - - - - -
3 - - - - - -
4 - - - - - -
5 - - - - - -
Tour 0
  0 1 2 3 4 5
0 - - - - - -
1 - - - - - -
2 - - - - - -
3 - - - - - -
4 - - - - - -
5 - - - - - -
new mtt: (0, 0), (5, 5)
mtt: {'top': 0, 'bottom': 5, 'left': 0, 'right': 5}
Tour 1
  0 1 2 3 4 5
0 - - - - - -
1 - - - - - -
2 - - X - - -
3 - - - - - -
4 - - - - - -
5 - - - - - -
entrez un nombre entre 0 et 53
entrez un nombre entre 0 et 52
new mtt: (0, 0), (5, 5)
mtt: {'top': 0, 'bottom': 5, 'left': 0, 'right': 5}
Tour 2
  0 1 2 3 4 5
0 - - - - - -
1 - - - - - -
2 - - X - - -
3 - - O - - -
4 - - - - - -
5 - - - - - -
bottom 5 right 5
0 0
0 0 0 4
0 1 0 5
0 2 0 5
0 3 0 5
0 4 0 4
0 5 0 5
1 0 0 5
1 1 0 4


KeyboardInterrupt: ignored

In [ ]:
i,j, size = 4, 1, 6
mtt["top"] = max(i - 3, 0)
mtt["left"] = max(j - 3, 0)
mtt["right"] = min(j + 3 - mtt["left"], size - 1)
mtt["bottom"] = min(i + 3 - mtt["top"], size - 1)
print(mtt["top"], mtt["bottom"], mtt["left"], mtt["right"])

In [ ]:
q4

In [ ]:
3

In [ ]:
1%6